In [1]:
import numpy as np

# 오차제곱합

In [2]:
def sum_squares_error(y,t):
  return 0.5 * np.sum((y-t)**2)

In [3]:
# 오차제곱합
## => 오차제곱합 기준으로 오차가 더 작은게 정답에 더 가깝다!
## 정답은 2
t = [0,0,1,0,0,0,0,0,0,0]
## 2일 확률이 가장 높다고 추정함
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
print(sum_squares_error(np.array(y), np.array(t)))
## 7일 확률이 가장 높다고 추정함
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
print(sum_squares_error(np.array(y), np.array(t)))

0.09750000000000003
0.5975


# 교차 엔트로피 오차

In [4]:
def cross_entropy_error(y,t):
  delta = 1e-7
  return -np.sum(t*np.log(y+delta)) # delta 더하는 이유: log0방지 ( log0은 음의 무한대 )

In [5]:
# 교차 엔트로피 오차
## =>교차 엔트로피 오차 기준에서도 오차가 더 작은게 정답에 더 가깝다!
## 정답은 2
t = [0,0,1,0,0,0,0,0,0,0]
## 2일 확률이 가장 높다고 추정함
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))
## 7일 확률이 가장 높다고 추정함
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


# 미니배치 학습

- 데이터 불러오기

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import sys, os
sys.path.append('/content/drive/MyDrive/DACON_사기탐지 ')
from mnist import load_mnist

In [8]:
(x_train, t_train) , (x_test, t_test) = load_mnist(one_hot_label = True, normalize = False)
print(x_train.shape) # (60000, 784)
print(t_train.shape) # (60000,)
print(x_test.shape) # (10000, 784)
print(t_test.shape) # (10000,)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


- 랜덤 지정
  - np.random.choice :  지정한 범위의 수 중에서 무작위로 원하는 개수만 꺼내기

In [9]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size , batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

# (배치용) 교차 엔트로피 오차 구현하기

In [10]:
def cross_entropy_error(y,t):
  if y.ndim ==1 :
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)

  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size

# 미분

In [11]:
def numerical_diff(f,x):
  h = 1e-50
  return (f(x+h)-f(x))/h

In [12]:
# 중앙 차분으로 개선
def numerical_diff(f,x):
  h = 1e-4
  return (f(x+h)-f(x-h))/(2*h)

In [13]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)  # x와 형상이 같은 배열을 생성

    for idx in range(x.size):
        tmp_val = x[idx]
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2 * h)
        x[idx] = tmp_val  # 값 복원

    return grad

# 경사하강법

In [14]:
def gradient_descent(f, init_x, lr = 0.01 , step_num = 100):
  x = init_x
  for i in range(step_num):
    grad = numerical_gradient(f,x)
    x -= lr* grad
  return x

## 문제 : 경사하강법으로 f(x0, x1) = x0**2 + x1**2의 최솟값을 구하라

In [15]:
def function_2(x):
  return x[0]**2+ x[1]**2

In [16]:
init_x = np.array([-3.0 , 4.0])
gradient_descent(function_2, init_x = init_x, lr = 0.1, step_num = 100)

array([-6.11110793e-10,  8.14814391e-10])

# 신경망의 기울기

In [18]:
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

In [19]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)  # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [20]:
net = simpleNet()
print(net.W) # 가중치 매개변수

[[-0.52058041 -0.73017004  0.62198633]
 [ 0.40716329 -0.34643643  1.63328252]]


In [21]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[ 0.05409872 -0.74989481  1.84314607]


In [22]:
np.argmax(p)

2

In [23]:
t = np.array([0,0,1])
net.loss(x,t)

0.21665164809526016

In [24]:
def f(W):
    return net.loss(x,t)
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.0807397   0.03613409 -0.1168738 ]
 [ 0.12110955  0.05420114 -0.17531069]]


# 2층 신경망 클래스 구현

In [25]:
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)

        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

# 미니배치 학습 구현

In [ ]:
(x_train, t_train) , (x_test, t_test) = load_mnist(one_hot_label = True, normalize = False)

train_loss_list= []

# 하이퍼파라미터
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

for i in range(iters_num):

    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)  # 다음 장에서 구현할 더 빠른 방법!

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)



In [ ]:
# 그래프 그리기
import matplotlib.pyplot as plt
plt.plot(train_loss_list)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# 시험데이터로 평가하기

In [ ]:

train_loss_list= []
train_acc_list = []
test_acc_list = []

# 하이퍼파라미터
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

# 1에폭당 반복수
iter_per_epoch = max(train_size / batch_size , 1)

for i in range(iters_num):

    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    # grad = network.gradient(x_batch, t_batch)  # 다음 장에서 구현할 더 빠른 방법!

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

        # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "
              + str(train_acc) + ", " + str(test_acc))

